In [1]:
%autosave 0
# this disables autosave for instructor solutions
# if a student makes changes to the instructor solutions then the instructor pushes updates this can cause a git conflict

# Import CSV Data Into PyMongo

In [26]:
from pymongo import MongoClient
import csv
from pprint import pprint
from datetime import datetime
from flask import jsonify

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'electric_charging', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# assign the database to a variable name
db = mongo['electric_charging']
stations = db['stations']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['stations']


In [18]:
db.drop_collection('stations')

{'nIndexesWas': 1, 'ns': 'electric_charging.stations', 'ok': 1.0}

In [19]:
with open('test_data.csv', 'r', encoding='utf-8-sig') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        station = {k: v for k, v in row.items() if v}
        
        if 'Station Phone' in station:
            station['Station Phone'] = row['Station Phone'].split()
        if 'EV Connector Types' in station:
            station['EV Connector Types'] = station['EV Connector Types'].split()
            
        open_date = datetime.strptime(station['Open Date'], "%m/%d/%Y")
        last_confirmed = datetime.strptime(station['Date Last Confirmed'], "%m/%d/%Y")
        station['Open Date'] = open_date.strftime("%Y-%m-%d")
        station['Date Last Confirmed'] = last_confirmed.strftime("%Y-%m-%d")
        
        stations.insert_one(station)

In [20]:
# review a document in the customer_list collection
pprint(db.stations.find_one())

{'Access Days Time': '5:30am-9pm; pay lot',
 'City': 'Los Angeles',
 'Date Last Confirmed': '2023-01-10',
 'EV Connector Types': ['J1772'],
 'EV Level2 EVSE Num': '7',
 'EV Pricing': 'Free; parking fee',
 'Facility Type': 'PARKING_GARAGE',
 'Latitude': '34.040539',
 'Longitude': '-118.271387',
 'Open Date': '1995-08-30',
 'Owner Type Code': 'P',
 'State': 'CA',
 'Station Name': 'Los Angeles Convention Center',
 'Station Phone': ['213-741-1151'],
 'Street Address': '1201 S Figueroa St',
 'ZIP': '90015',
 '_id': ObjectId('67983429af229cce568a2d5f')}


In [21]:
stations.update_many({}, [ {'$set': {
    "EV Level1 EVSE Num" : {'$toInt': "$EV Level1 EVSE Num"},
    "EV Level2 EVSE Num" : {'$toInt': "$EV Level2 EVSE Num"},
    "EV DC Fast Count" : {'$toInt': "$EV DC Fast Count"},
    "Latitude" : {'$toDouble': "$Latitude"},
    "Longitude" : {'$toDouble': "$Longitude"}
}} ])

UpdateResult({'n': 59, 'nModified': 59, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [27]:
jsonify(list(db.stations.find({})))

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.